In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from fake_useragent import UserAgent
import os


In [ ]:
import os
import csv
import cloudscraper
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
from fake_useragent import UserAgent

In [1]:

def fetch_phones_from_api():
    scraper = cloudscraper.create_scraper()  # Tạo cloudscraper để vượt qua Cloudflare
    url = "https://api.cellphones.com.vn/v2/graphql/query"
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "origin": "https://cellphones.com.vn",
        "referer": "https://cellphones.com.vn/",
        "user-agent": UserAgent().random,  # Random User-Agent
    }
    page = 1
    page_size = 200
    all_phones = []

    while True:
        query = {
            "query": f"""
            query GetProductsByCateId {{
                products(
                    filter: {{
                        static: {{
                            province_id: 30,
                            stock: {{ from: 0 }},
                            categories: ["3"]  # CategoryId của đt di động
                        }},
                        dynamic: {{}}
                    }},
                    page: {page},
                    size: {page_size},
                    sort: [{{ view: desc }}]
                ) {{
                    general {{
                        product_id
                        name
                        url_path
                    }}
                    filterable {{
                        price
                        special_price
                    }}
                }}
            }}
            """,
            "operationName": "GetProductsByCateId"
        }

        response = scraper.post(url, headers=headers, json=query)
        if response.status_code != 200:
            print(f"Failed to fetch products. Status Code: {response.status_code}")
            print(response.text)
            break

        data = response.json()
        products = data.get("data", {}).get("products", [])
        if not products:
            break

        for product in products:
            all_phones.append({
                "name": product["general"]["name"],
                "price": product["filterable"].get("price", "N/A"),
                "special_price": product["filterable"].get("special_price", "N/A"),
                "url": "https://cellphones.com.vn/" + product["general"]["url_path"]
            })

        page += 1

    return all_phones

class CellphonesCrawler:
    def __init__(self):
        self.chrome_driver_path = r"C:\\Users\\Quang Anh\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"
        os.environ["webdriver.chrome.driver"] = self.chrome_driver_path
        self.options = Options()
        self.options.add_argument("--disable-blink-features=AutomationControlled")
        self.options.add_argument("--headless")  # Chạy trình duyệt ở chế độ headless
        self.options.add_argument("--disable-extensions")
        self.options.add_argument("--no-sandbox")
        self.options.add_argument("--disable-dev-shm-usage")
        self.options.add_argument("--disable-images")  # Tắt tải hình ảnh
        self.options.add_argument("start-maximized")
        #user agent
        self.options.add_argument(f"user-agent={UserAgent().random}")

    def init_driver(self):
        self.driver = webdriver.Chrome(service=Service(self.chrome_driver_path), options=self.options)

        # Áp dụng selenium-stealth
        stealth(
            self.driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )

    def close_driver(self):
        if self.driver:
            self.driver.quit()

    def scrape_product(self, url):
        print(f"Đang truy cập URL sản phẩm: {url}")
        self.driver.get(url)

        specs_dict = {}
        try:
            # Sử dụng JavaScript để lấy nhanh thông số kỹ thuật
            script = """
            const specs = {};
            document.querySelectorAll('li.technical-content-item').forEach(item => {
                const key = item.querySelector('p')?.innerText.trim();
                const value = item.querySelector('div')?.innerText.trim();
                if (key && value) specs[key] = value;
            });
            return specs;
            """
            specs_dict = self.driver.execute_script(script)
        except Exception as e:
            print(f"Lỗi khi lấy thông số kỹ thuật bằng JavaScript: {e}")

        return specs_dict

    def crawl(self, phones, output_file):
        self.init_driver()

        try:
            all_specs = set()  # Tập hợp các thông số kỹ thuật duy nhất
            phone_data = []  # Lưu trữ dữ liệu sản phẩm

            # Crawl dữ liệu
            for phone in phones:
                try:
                    specs = self.scrape_product(phone['url'])
                    phone.update(specs)
                    all_specs.update(specs.keys())  # Thêm thông số vào tập hợp
                    phone_data.append(phone)
                except Exception as e:
                    print(f"Lỗi khi xử lý sản phẩm {phone.get('name', 'N/A')}: {e}")

            # Chuẩn bị header CSV
            all_specs = sorted(all_specs)
            headers = ["Tên sản phẩm", "Giá", "Giá khuyến mãi", "URL"] + list(all_specs)
            print(f"Header được tạo: {headers}")

            # Ghi file CSV
            with open(output_file, mode='w', encoding='utf-8-sig', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=headers)
                writer.writeheader()  # Ghi header

                for phone in phone_data:
                    # Đổi tên khóa cho phù hợp với header
                    row = {
                        "Tên sản phẩm": phone.get("name", "N/A"),
                        "Giá": phone.get("price", "N/A"),
                        "Giá khuyến mãi": phone.get("special_price", "N/A"),
                        "URL": phone.get("url", "N/A")
                    }
                    # Thêm thông số kỹ thuật
                    row.update({key: phone.get(key, "N/A") for key in all_specs})
                    writer.writerow(row)

            print(f"Đã lưu dữ liệu vào {output_file}")
        finally:
            self.close_driver()


if __name__ == "__main__":
    phones = fetch_phones_from_api()
    print(f"Số lượng sản phẩm từ API: {len(phones)}")
    print(f"Số lượng sản phẩm để crawl: {len(phones)}")

    crawler = CellphonesCrawler()
    crawler.crawl(phones, "cellphones_data.csv")


Số lượng sản phẩm từ API: 1939
Số lượng sản phẩm để crawl: 1939
Đang truy cập URL sản phẩm: https://cellphones.com.vn/iphone-16-pro-max.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/dien-thoai-xiaomi-redmi-note-14.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/dien-thoai-samsung-galaxy-s24-fe.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/dien-thoai-oppo-reno13f-4g.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/iphone-15-pro-max.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/iphone-16.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/samsung-galaxy-s24-ultra.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/samsung-galaxy-z-flip-6.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/dien-thoai-samsung-galaxy-m55.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/dien-thoai-samsung-galaxy-a16.html
Đang truy cập URL sản phẩm: https://cellphones.com.vn/iphone-13.html
Đang truy cập URL sản phẩm: ht

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('cellphones_data.csv')
data

,Tên sản phẩm,Giá,Giá khuyến mãi,URL,Bộ nhớ trong,Camera sau,Camera trước,Chipset,Công nghệ NFC,Công nghệ màn hình,Cảm biến,Dung lượng RAM,Hệ điều hành,Kích thước màn hình,Loại CPU,Pin,Thẻ SIM,Tính năng màn hình,Tương thích,Độ phân giải màn hình
0,iPhone 16 Pro Max 256GB | Chính hãng VN/A,34990000.0,32990000.0,https://cellphones.com.vn/iphone-16-pro-max.html,256 GB,"Camera chính: 48MP, f/1.78, 24mm, 2µm, chống r...","12MP, ƒ/1.9, Tự động lấy nét theo pha Focus Pi...",Apple A18 Pro,Có,Super Retina XDR OLED,NaN,NaN,iOS 18,6.9 inches,CPU 6 lõi mới với 2 lõi hiệu năng và 4 lõi hiệ...,NaN,Sim kép (nano-Sim và e-Sim) - Hỗ trợ 2 e-Sim,Dynamic IslandMàn hình Luôn Bật Công nghệ ProM...,Tương Thích Với Thiết Bị Trợ Thính,2868 x 1320 pixels
1,Xiaomi Redmi Note 14 6GB-128GB,4990000.0,0.0,https://cellphones.com.vn/dien-thoai-xiaomi-re...,128 GB,Hệ thống camera AI 108MP\nCamera đo chiều sâu ...,Camera trước 20MP - f/2.2,MediaTek Helio G99-Ultra,Có,AMOLED,NaN,6 GB,Android 14,6.67 inches,"8 nhân, lên đến 2.2GHz",5500mAh,2 Nano-SIM,Tốc độ làm mới: Lên đến 120Hz\nĐộ sáng: 1800 n...,NaN,1080 x 2400 pixels (FullHD+)
2,Samsung Galaxy S24 FE 5G 8GB 128GB,16990000.0,13990000.0,https://cellphones.com.vn/dien-thoai-samsung-g...,128 GB,50MP + 12vMP + 8MP,10MP,Exynos 2400e (4nm),NaN,Dynamic AMOLED,NaN,8 GB,Android 14,6.7 inches,NaN,4700 mAh,NaN,120Hz,NaN,1080 x 2400 pixels (FullHD+)
3,OPPO Reno13 F 8GB-256GB,8990000.0,0.0,https://cellphones.com.vn/dien-thoai-oppo-reno...,256 GB,"Camera chính: 50MP, F/1.8\nCamera góc rộng: 8M...","32MP, f/2.4","MediaTek Helio G100, tối đa 2.2GHz",Có,AMOLED,NaN,8 GB,"ColorOS 15, nền tảng Android 15",6.67 inches,NaN,5800mAh (Typ),2 SIM (Nano-SIM),"120Hz, Kính cường lực AGC DT Star2\n1200 nits\...",NaN,1080 x 2400 pixels (FullHD+)
4,iPhone 15 Pro Max 256GB | Chính hãng VN/A,34990000.0,29290000.0,https://cellphones.com.vn/iphone-15-pro-max.html,256 GB,"Camera chính: 48MP, 24 mm, ƒ/1.78 Camera góc s...","12MP, ƒ/1.9",Apple A17 Pro 6 nhân,Có,Super Retina XDR OLED,NaN,8 GB,iOS 17,6.7 inches,CPU 6 lõi mới với 2 lõi hiệu năng và 4 lõi hiệ...,4422 mAh,2 SIM (nano‑SIM và eSIM),Tốc độ làm mới 120Hz460 ppi2000 nitsHDRTrue To...,NaN,2796 x 1290-pixel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,Vivo Y71,0.0,0.0,https://cellphones.com.vn/vivo-y71.html,16 GB,13 MP,5 MP,Snapdragon 425 4 nhân,Không,IPS LCD,NaN,3 GB,NaN,6.22 inches,NaN,3360 mAh,2 SIM (Nano-SIM),Mặt kính cong 2.5D,NaN,720 x 1440 pixels
1935,TECNO CAMON 40 12GB 256GB,0.0,0.0,https://cellphones.com.vn/dien-thoai-tecno-cam...,256 GB,NaN,NaN,NaN,NaN,NaN,NaN,12 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,Xiaomi Redmi Note 14 Pro Plus 8GB 128GB,0.0,0.0,https://cellphones.com.vn/dien-thoai-xiaomi-re...,128 GB,NaN,NaN,NaN,NaN,NaN,NaN,8 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1937,Điện thoại Itel RS4 12GB 256GB NFC,0.0,0.0,https://cellphones.com.vn/dien-thoai-itel-rs4-...,256 GB,"Camera góc rộng: 50 MP, f/1.6, AF",8 MP,Mediatek Helio G99 Ultimate,Có,IPS LCD,NaN,12 GB,Android 13,6.56 inches,2x2.2 GHz Cortex-A76 & 6x2.0 GHz Cortex-A55,"5,000 mAh",2 Nano-SIM,Tần số quét 120HzTỷ lệ 20:9 (mật độ ~ 269 ppi),NaN,720 x 1612 (HD+)


In [22]:
pip install selenium-stealth


  Obtaining dependency information for selenium-stealth from https://files.pythonhosted.org/packages/cb/ac/7877df8b819d54a4e317a093a0a9e0a38d21d884a7250aa713f2f0869442/selenium_stealth-1.0.6-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install cloudscraper


  Obtaining dependency information for cloudscraper from https://files.pythonhosted.org/packages/81/97/fc88803a451029688dffd7eb446dc1b529657577aec13aceff1cc9628c5d/cloudscraper-1.2.71-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/99.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/99.7 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/99.7 kB ? eta -:--:--
   ------------------------ --------------- 61.4/99.7 kB 550.5 kB/s eta 0:00:01
   ---------------------------------------- 99.7/99.7 kB 715.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
